# Исследование надёжности заёмщиков

## Шаг 1. Обзор данных

In [ ]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [21]:
display(df.head(13))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов. Тип данных — `object`, `int` и `float`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — есть ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях колонок нарушений стиля нет.
Количество значений в столбцах `days_employed` и `total_income` отличается от остальных и имеют тип данных `float`. Значит, в данных есть пропущенные значения. В столбце `days_employed`, значения имеют отрицательные значения, при этом строки, которые имеют положительные значения, очень большие. Стаж не может быть больше 100 лет. Предположительно, положительные значения указаны не в днях, а часах. 

В данных есть символы в разном регистре, в колонке с целью получения кредита нет единообразия. 

### Вывод
В каждой строке таблицы - данные о заемщике банка. Столбцы описывают колличество детей, семейное положение, среднемесячный доход и другое.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски, данные в разных размерностях в одном столбце, отрицательные значения, значения в разном регистре. 

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [22]:
#Обработка артефактов
#Судя по таблице, в колонке 'days_employed' данные в днях имеют отрицательное значение, 
#а положительные данные имеют неправдоподобно большое значение - следовательно, записаны в часах.
df.loc[df['days_employed']>0, 'days_employed'] = df.loc[df['days_employed']>0, 'days_employed'] / 24
df['days_employed'] = df['days_employed'].apply(abs)
display(df.head(13))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [24]:
#Обработка пропусков медианным значением по группировке

print(df.isna().sum())

def fill_median_total(df):
    income_medians = df.groupby('income_type').median()['total_income']
    for income_type in income_medians.index:
        change_ix = (df['income_type'] == income_type) & (df['total_income'].isna())
        print(change_ix)
        df.loc[change_ix, 'total_income'] = income_medians[income_type]    
    return df.isna().sum()

print(fill_median_total(df))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64
0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21525, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21525, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21525, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521 

In [4]:
def fill_median_type(df):
    income_medians = df.groupby('income_type').median()['days_employed']
    for income_type in income_medians.index:
        change_ix = (df['income_type'] == income_type) & (df['days_employed'].isna())
        df.loc[change_ix, 'days_employed'] = income_medians[income_type]    
    return df.isna().sum()

print(fill_median_type(df)) #проверка отсутствия  пропусков в данных 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

Произведена обработка артефактов - отрицательные значения заменены на положительные, данные в часах переведены в дни. Найденные пропуски могут помешать проверке только одной из четырех гипотез. Однако, пропуски составляют лишь одну десятую часть всех данных. В данном случае заменила пропуски на значения медианы по группировке income_type. Предполагаемые причины пропусков точно установить в условиях тренажера невозможно. Но можно предположить, что причина пропусков значений - ошибка в выгрузке данных. Так как такую информацию, как дни стажа и среднемесячный доход, невозможно не указать при получении кредита. 

### Замена типа данных

In [6]:
columns_to_replace = ['days_employed', 'total_income']
for column in columns_to_replace:
    df[column] = df[column].astype('int')
df.info() #проверка типов данных после обработки

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
 12  income_type_id    21525 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


### Вывод

Произведена замена вещественного типа данных на целочисленный методом 'astype'.

### Обработка дубликатов

In [7]:
print('Количество дубликатов:', df.duplicated().sum())  #получение колличества дубликатов

df['education'] = df['education'].str.lower() #приведение к нижнему регистру в столбце

print('Количество дубликатов после приведения к нижнему регистру:', df.duplicated().sum()) #повторное получение колличества дубликатов, видим, что оно увеличилось 

df = df.drop_duplicates().reset_index(drop = True)
print('Количество дубликатов после удаления:', df.duplicated().sum()) #проверка, что все дубликаты удалены

Количество дубликатов: 54
Количество дубликатов после приведения к нижнему регистру: 71
Количество дубликатов после удаления: 0


### Вывод

В ходе удаления явных дубликатов выяснено, что при приведении к нижнему регистру значений в столбце 'education', количество дубликатов увеличилось. Явные дубликаты удалены методом drop_duplicates() с перезаписью индексов. На мой взгляд, причиной появления дубликатов в данном случае, может быть неверное соединение данных из разных источников. Эта гипотеза подкрепляется тем, что ранее мы видели в столбце со стажем, что одна часть данных имеет отрицательное значение и записана в днях, а другая положительное, но записана в часах. Очевидно слияние минимум из 2-х источников.

### Лемматизация

In [8]:
#Получение строки с уникальными значениями
uni = df['purpose'].sort_values().unique()
uni_str = ' '.join(uni)

m = Mystem()
lemmas = m.lemmatize(uni_str)

display(Counter(lemmas))
    
def categorize(purpose):
    lemmas = m.lemmatize(purpose)
    
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'прочее'


df['category_purpose'] = df['purpose'].apply(categorize)
display(df.head())
#print(df['category'].sort_values().unique())

report = df.groupby('category_purpose').agg({'category_purpose': 'count'})
report['%'] = (report['category_purpose']/report['category_purpose'].sum())*100
display(report)

Counter({'автомобиль': 9,
         ' ': 96,
         'высокий': 3,
         'образование': 9,
         'дополнительный': 2,
         'жилье': 7,
         'заниматься': 2,
         'на': 4,
         'покупка': 10,
         'подержать': 2,
         'свой': 4,
         'проведение': 1,
         'свадьба': 3,
         'недвижимость': 10,
         'операция': 4,
         'с': 5,
         'коммерческий': 2,
         'со': 1,
         'жилой': 2,
         'для': 2,
         'сдача': 1,
         'семья': 1,
         'получение': 3,
         'приобретение': 1,
         'профильный': 1,
         'ремонт': 1,
         'сделка': 2,
         'строительство': 3,
         'собственный': 1,
         'сыграть': 1,
         '\n': 1})

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба


,category_purpose,%
category_purpose,,
автомобиль,4306,20.070849
недвижимость,10811,50.391535
образование,4013,18.705137
свадьба,2324,10.832479


### Вывод

Проведена лемматизация столбца с целью кредита, категории цели кредита выведены в отдельный столбец. Получена информация, что больше всего кредитов берут для покупки недвижимости. Далее идут, практически в равных долях, кредиты на покупку автомобиля и образование, меньше всего берут кредиты на свадьбы. 

### Категоризация данных

In [9]:
children = df[['children']].drop_duplicates()
display(children)

#замена непровдаподобных значений
df.loc[df['children'] == -1, 'children'] = 1 
df.loc[df['children'] == 20, 'children'] = 2
        
children = df[['children']].drop_duplicates()
display(children)

#категоризация
def category_children(child):
    if child == 0:
        return 'нет детей'
    if child < 3:
        return 'есть дети'
    return 'многодетные'

children['category_chi'] = df['children'].apply(category_children)
display(children)

df['category_chi'] = df['children'].apply(category_children)
display(df.head())

,children
0,1
2,0
3,3
8,2
291,-1
294,4
606,20
3977,5


,children
0,1
2,0
3,3
8,2
294,4
3977,5


,children,category_chi
0,1,есть дети
2,0,нет детей
3,3,многодетные
8,2,есть дети
294,4,многодетные
3977,5,многодетные


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,category_purpose,category_chi
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование,многодетные
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба,нет детей


In [10]:
display(df[['total_income']].drop_duplicates())

#для категоризации необходимо знать пороговые и медианное значение
mini = df['total_income'].min()
print(mini)
med = df['total_income'].median()
print(med)
maxi = df['total_income'].max()
print(maxi)

total_income = df[['total_income']].drop_duplicates()
display(total_income)

#категоризация
#выше бедности <= 40000
#40000 < ниже среднего <= 100000
#100000 < нижний средний <= 150000
#150000 < средний <= 250000
#250000 < обеспеченныe <= 500000
#богатые > 500000 

def category_income(income):
    if income <= 40000:
        return 'выше бедности'
    if 40000 < income <= 100000:
        return 'ниже среднего'
    if 100000 < income <= 150000:
        return 'нижний средний'
    if 150000 < income <= 250000:
        return 'средний'
    if 250000 < income <= 500000:
        return 'обеспеченныe'
    return 'богатые'

df['category_income'] = df['total_income'].apply(category_income)
display(df.head())

,total_income
0,253875
1,112080
2,145885
3,267628
4,158616
...,...
21449,224791
21450,155999
21451,89672
21452,244093


20667
142594.0
2265604


,total_income
0,253875
1,112080
2,145885
3,267628
4,158616
...,...
21449,224791
21450,155999
21451,89672
21452,244093


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,category_purpose,category_chi,category_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость,есть дети,обеспеченныe
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль,есть дети,нижний средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость,нет детей,нижний средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование,многодетные,обеспеченныe
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба,нет детей,средний


In [11]:
#просмотр уникальных значений и выделение словарей
display(df[['days_employed']].drop_duplicates())
display(df[['dob_years']].drop_duplicates())
display(df[['education', 'education_id']].drop_duplicates())
display(df[['family_status', 'family_status_id']].drop_duplicates())
display(df[['income_type']].drop_duplicates())
display(df[['debt']].drop_duplicates())
display(df[['purpose', 'category_purpose']].drop_duplicates())

,days_employed
0,8437
1,4024
2,5623
3,4124
4,14177
...,...
21402,9945
21409,8799
21423,960
21428,9929


,dob_years
0,42
1,36
2,33
3,32
4,53
5,27
6,43
7,50
8,35
9,41


,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


,income_type
0,сотрудник
4,пенсионер
5,компаньон
26,госслужащий
3132,безработный
5931,предприниматель
9399,студент
20778,в декрете


,debt
0,0
14,1


,purpose,category_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость
10,покупка недвижимости,недвижимость
11,покупка коммерческой недвижимости,недвижимость


### Вывод

В таблице присутствуют словари, они были выделены. Для удобства дальнейшей работы, были выделены категории в столбцах с количеством детей и среднемесячным заработком, столбцы с ними были добавлены в таблицу. На следующем этапе проекта будет удобно группировать по ним и вычислять процент отсутствия задолженности. 

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
report_chi = df.groupby('category_chi').agg({'debt':['count', 'sum', 'mean']})
report_chi.columns = ['total', 'debt', '% debt']
report_chi['% debt'] = report_chi['% debt']*100
report_chi

,total,debt,% debt
category_chi,,,
есть дети,6983,647,9.265359
многодетные,380,31,8.157895
нет детей,14091,1063,7.543822


### Вывод

Меньше всего просрочек допускают люди без детей, больше всего просрочек у клиентов с 1-2 детьми. Как мы видим из таблицы, процент просрочек по кредитам увеличивается с появлением детей. Причем между бездетными и многодетными разница всего 0.62%. А между теми, у кого нет детей  и у кого 1-2 ребенка (есть дети) - 1.72%. Возможно, это связно с тем, что многодетные люди берут кредиты на много реже, чем бездетные и люди, у кого 1-2 ребенка. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
report_family = df.groupby('family_status').agg({'debt':['count', 'sum', 'mean']})
report_family.columns = ['total', 'debt', '% debt']
report_family['% debt'] = report_family['% debt']*100
report_family

,total,debt,% debt
family_status,,,
Не женат / не замужем,2810,274,9.750890
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182


### Вывод

Меньше всего просрочек по кредитам у вдовцов/вдов, но и берут кредиты они меньше всего. 
Практически аналогичная ситуация с людьми в разводе. Они берут кредиты чуть больше, но и просрочек немного больше. 
Больше всего кредитов берут женатые люди и имеют по ним сравнительно низкий процент просрочек.
Стоит обратить внимаение на неженатых клиентов и находящихся в гражданских браках, они имеют большой процент просрочек. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
report_income = df.groupby('category_income').agg({'debt':['count', 'sum', 'mean']})
report_income.columns = ['total', 'debt', '% debt']
report_income['% debt'] = report_income['% debt']*100
report_income

,total,debt,% debt
category_income,,,
богатые,222,14,6.306306
выше бедности,125,10,8.000000
ниже среднего,4338,344,7.929922
нижний средний,7160,624,8.715084
обеспеченныe,2591,180,6.947125
средний,7018,569,8.107723


### Вывод

Исходя из данных таблицы можно выдвинуть предположение, что уровень дохода влияет на возврат кредита, если речь идет о богатых и обеспеченных людях - они действительно допускают просрочки реже. Остальные категории доходов допускают просрочки примерно одинаково.  

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
report_purpose = df.groupby('category_purpose').agg({'debt':['count', 'sum', 'mean']})
report_purpose.columns = ['total', 'debt', '% debt']
report_purpose['% debt'] = report_purpose['% debt']*100
report_purpose

,total,debt,% debt
category_purpose,,,
автомобиль,4306,403,9.359034
недвижимость,10811,782,7.233373
образование,4013,370,9.220035
свадьба,2324,186,8.003442


### Вывод

Меньше всего просрочек по кредитам на недвижимость, что объяснимо, так как жилье - это основная потребность человека. На втором месте кредиты на свадьбы. Больше всего просрочек по кредитам на образование и автомобили, стоит обратить внимание на это. 

## Шаг 4. Общий вывод

Изучены 4 вопроса.

Есть ли зависимость между наличием детей и возвратом кредита в срок?
Действительно, это так. Меньше всего просрочек допускают заемщики без детей, больше всего просрочек у людей с 1-2 детьми. Многодетные люди берут кредиты намного реже, чем бездетные и люди, у кого есть 1-2 ребенка. 

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть,  самые надежные заемщики - люди в разводе и вдовцы. Небольшая разница с ними у женатых людей - эта самая многочисленная категория заемщиков. Самый большой процент просрочек у неженатых людей и живущих в гражданских браках, стоит обратить на это внимание. 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Зависимость есть, но не такая сильная. Богатые и обеспеченные люди являются самыми надежными заемщиками. Остальные категории доходов возвращают кредиты одинаково.  

Как разные цели кредита влияют на его возврат в срок?
Меньше всего просрочек по кредитам на недвижимость - это оъяснимо, так как это самые обеспеченные кредиты. На втором месте кредиты на свадьбы. Больше всего просрочек по кредитам на образование и автомобили, это плохо обеспеченные кредиты, так как образование не гарантирует получение средств, а автомобиль может быть испорчен и сильно потерять в цене.